In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

import torch
from nanochat.gpt import GPT, GPTConfig

# 1. 定义配置 (这里使用一个小模型作为演示)
config = GPTConfig(
    vocab_size=50304, # 词表大小
    sequence_len=1024,# 最大上下文长度
    n_layer=4,        # Transformer 层数
    n_head=4,         # 注意力头数
    n_kv_head=4,   
    n_embd=128        # 嵌入维度
)

# 2. 实例化模型
model = GPT(config)

# 3. 初始化权重 (关键步骤)
# 应用特定的正态分布初始化，并将 lm_head 和部分投影层置零
model.init_weights()

print(f"模型参数量: {sum(p.numel() for p in model.parameters())}")

# --- 场景 A: 推理 (Inference) ---
# 构造模拟输入: Batch Size=2, Sequence Length=16
B, T = 2, 5
idx = torch.randint(0, config.vocab_size, (B, T))

# 前向传播 (不传 targets)
logits = model(idx)

# 输出形状: (Batch, Seq_Len, Vocab_Size)
print(f"\n输入形状: {idx.shape}")
print(f"Logits 形状: {logits.shape}") # -> torch.Size([2, 16, 50304])


# --- 场景 B: 训练 (Training) ---
# 构造模拟目标 (通常是 idx 向后移一位)
targets = torch.randint(0, config.vocab_size, (B, T))

# 前向传播 (传入 targets)
loss = model(idx, targets=targets)

# 输出是一个标量 Loss
print(f"\nTraining Loss: {loss.item():.4f}")

模型参数量: 13664256

输入形状: torch.Size([2, 5])
Logits 形状: torch.Size([2, 5, 50304])

Training Loss: 10.8258


In [2]:
idx

tensor([[50225,  8774,  1447,  6861, 46644, 25897, 39148, 21490, 15152, 29623,
          1377, 18267, 20870, 17969, 33869, 40159],
        [41792, 36984, 17243, 16975, 12120, 11724, 27727, 35220, 29049, 11822,
          5502, 20497, 14346, 23403, 19113, 21961]])

In [ ]:
next_token = logits[:, -1, :]
print(f"next_token形状: {next_token.shape}")

In [1]:
import os, sys
sys.path.append(os.path.abspath(".."))

from nanochat.tokenizer import get_tokenizer

text = "Weng earns $12 an hour for babysitting. Yesterday, she just did 50 minutes of babysitting. How much did she earn?"

tok = get_tokenizer()

# 1) 仅对这段 user 文本做 tokenize
ids = tok.encode(text)  # list[int]
tokens = [tok.id_to_token(i) for i in ids]  # list[str], 每个元素就是一个 token 的“字符串形态”
print("NUM_TOKENS:", len(tokens))
print(tokens)  # 如果太长你也可以逐行 print

# 2) 你的整条 GSM8K conversation 的“完整 token 流”
conv = {
  "messages": [
    {"role": "user", "content": text},
    {"role": "assistant", "content": [
      {"type": "text", "text": "Weng earns 12/60 = "},
      {"type": "python", "text": "12/60"},
      {"type": "python_output", "text": "0.2"},
      {"type": "text", "text": " per minute. Working 50 minutes, she earned 0.2*50 = "},
      {"type": "python", "text": "0.2*50"},
      {"type": "python_output", "text": "10"},
      {"type": "text", "text": ".\n#### 10"},
    ]},
  ]
}

full_ids, mask = tok.render_conversation(conv)
full_tokens = [tok.id_to_token(i) for i in full_ids]
print("FULL_NUM_TOKENS:", len(full_tokens))
print(full_tokens)

# 可选：同时打印 (token, id, mask) 方便核对哪些 token 来自 user/assistant/tool output
triples = [(repr(t), i, m) for t, i, m in zip(full_tokens, full_ids, mask)]
print(triples)

NUM_TOKENS: 33
['W', 'eng', ' earns', ' $', '12', ' an', ' hour', ' for', ' bab', 'ys', 'itting', '.', ' Y', 'ester', 'day', ',', ' she', ' just', ' did', ' ', '50', ' minutes', ' of', ' bab', 'ys', 'itting', '.', ' How', ' much', ' did', ' she', ' earn', '?']
FULL_NUM_TOKENS: 92
['<|bos|>', '<|user_start|>', 'W', 'eng', ' earns', ' $', '12', ' an', ' hour', ' for', ' bab', 'ys', 'itting', '.', ' Y', 'ester', 'day', ',', ' she', ' just', ' did', ' ', '50', ' minutes', ' of', ' bab', 'ys', 'itting', '.', ' How', ' much', ' did', ' she', ' earn', '?', '<|user_end|>', '<|assistant_start|>', 'W', 'eng', ' earns', ' ', '12', '/', '60', ' =', ' ', '<|python_start|>', '12', '/', '60', '<|python_end|>', '<|output_start|>', '0', '.', '2', '<|output_end|>', ' per', ' minute', '.', ' Working', ' ', '50', ' minutes', ',', ' she', ' earned', ' ', '0', '.', '2', '*', '50', ' =', ' ', '<|python_start|>', '0', '.', '2', '*', '50', '<|python_end|>', '<|output_start|>', '10', '<|output_end|>', '.\n', '#